### Definição de Constantes e Parâmetros

Nesta seção, definiremos as propriedades e dimensão dos elementos estruturais.
- Módulo de Elasticidade: Mede a rigidez do material. É definido como a razão entre a tensão aplicada e deformação sofrida pelo material. $$[\frac{Tensão}{Deforma\c{c}ão}] = N/m^2 = Pa$$.
- Área de Seção Transversal: Área da seção transversal dos elementos, definida como $$Comprimento_{externo}^2 - Comprimento_{Interno}^2,$$ pois todos os elementos possuem como seção transversal quadrados ocos. $$[Área \ da \ se\c{c}ão \ transversal] = m^2$$
- Comprimento: Distância entre dois nós da estrutura. $$[Comprimento] = m$$
- Ângulo: Orientação dos elementos em relação ao sistema de coordenadas globais. $$[Ângulo] = \frac{m}{m} = rad$$

In [227]:
import numpy as np
from sympy import *
np.set_printoptions(linewidth=200)
from sympy import init_printing
init_printing(num_columns=100)

# Definição de Constantes e Parâmetros
E = Symbol('E')
A = Symbol('A')
L1 = Symbol('L1')
L2 = Symbol('L2')
ang1 = Symbol('ang1')
ang2 = Symbol('ang2')
# Módulo de elasticidade (Pa)
E = 70e9

# Área da seção transversal (m²)
A = 0.000063

# Comprimentos dos elementos (m)
L1 = sqrt(5)  # Comprimento do elemento 1
L2 = 2           # Comprimento do elemento 2

# Ângulos dos elementos (radianos)
ang1 = atan(-0.5)  # Ângulo do elemento 1
ang2 = 0                # Ângulo do elemento 2

### Cálculo das Matrizes de Rigidez

$$
  \frac{EA}{L}
  \begin{bmatrix}
    \cos^2{\alpha}            & \cos{\alpha}\sin{\alpha}  & -\cos^2{\alpha}           & -\cos{\alpha}\sin{\alpha} \\
    \cos{\alpha}\sin{\alpha}  & \sin^2{\alpha}            & -\cos{\alpha}\sin{\alpha} & -\sin^2{\alpha}           \\
    -\cos^2{\alpha}           & -\cos{\alpha}\sin{\alpha} & \cos^2{\alpha}            & \cos{\alpha}\sin{\alpha}  \\
    -\cos{\alpha}\sin{\alpha} & -\sin^2{\alpha}            & \cos{\alpha}\sin{\alpha}  & \sin^2{\alpha}           \\
  \end{bmatrix}
$$
Essa é a matriz de rigidez das treliças 2D no Sistema Global de Coordenadas

In [228]:
# Constantes de rigidez axial dos elementos
EAKL1 = E * A / L1  # Rigidez axial do elemento 1
EAKL2 = E * A / L2  # Rigidez axial do elemento 2

# Matriz de Rigidez
def calcular_matriz_rigidez(E, A, L, angulo):
    EA_L = E * A / L
    c = cos(angulo)
    s = sin(angulo)
    return EA_L * Matrix([
        [c**2, c*s, -c**2, -c*s],
        [c*s, s**2, -c*s, -s**2],
        [-c**2, -c*s, c**2, c*s],
        [-c*s, -s**2, c*s, s**2]
    ])

# Matriz de rigidez do elemento 1
k1 = calcular_matriz_rigidez(E, A, L1, ang1)

# Matriz de rigidez do elemento 2
k2 = calcular_matriz_rigidez(E, A, L2, ang2)


print("Matriz de rigidez do elemento 1 (k1):")
pprint(k1)

print("\nMatriz de rigidez do elemento 2 (k2):")
pprint(k2)

Matriz de rigidez do elemento 1 (k1):
⎡705600.0⋅√5   -352800.0⋅√5  -705600.0⋅√5  352800.0⋅√5 ⎤
⎢                                                      ⎥
⎢-352800.0⋅√5  176400.0⋅√5   352800.0⋅√5   -176400.0⋅√5⎥
⎢                                                      ⎥
⎢-705600.0⋅√5  352800.0⋅√5   705600.0⋅√5   -352800.0⋅√5⎥
⎢                                                      ⎥
⎣352800.0⋅√5   -176400.0⋅√5  -352800.0⋅√5  176400.0⋅√5 ⎦

Matriz de rigidez do elemento 2 (k2):
⎡2205000.0   0  -2205000.0  0⎤
⎢                            ⎥
⎢    0       0      0       0⎥
⎢                            ⎥
⎢-2205000.0  0  2205000.0   0⎥
⎢                            ⎥
⎣    0       0      0       0⎦


### Montagem da Matriz Global

In [229]:
def montar_matriz_global(k1, k2):
    # Inicialização da matriz global de rigidez (6x6) com zeros
    matriz_global = Matrix.zeros(6, 6)
    # Inserção da matriz de rigidez do elemento 1 na matriz global
    matriz_global[0:4, 0:4] += k1
    # Inserção da matriz de rigidez do elemento 2 na matriz global
    matriz_global[2:6, 2:6] += k2
    
    return matriz_global

matriz_global = montar_matriz_global(k1, k2)

print("Matriz global de rigidez:")
pprint(matriz_global)

Matriz global de rigidez:
⎡705600.0⋅√5   -352800.0⋅√5       -705600.0⋅√5        352800.0⋅√5       0       0⎤
⎢                                                                                ⎥
⎢-352800.0⋅√5  176400.0⋅√5         352800.0⋅√5        -176400.0⋅√5      0       0⎥
⎢                                                                                ⎥
⎢-705600.0⋅√5  352800.0⋅√5   705600.0⋅√5 + 2205000.0  -352800.0⋅√5  -2205000.0  0⎥
⎢                                                                                ⎥
⎢352800.0⋅√5   -176400.0⋅√5       -352800.0⋅√5        176400.0⋅√5       0       0⎥
⎢                                                                                ⎥
⎢     0             0              -2205000.0              0        2205000.0   0⎥
⎢                                                                                ⎥
⎣     0             0                   0                  0            0       0⎦


### Resolução do Sistema

In [230]:
# Redução da matriz global para considerar as condições de contorno
vetor_forcas_reduzido = Matrix([0, -1000])  # Força aplicada no nó correspondente

def resolucao_sistema(matriz_global, vetor_forcas_conhecido):
    # Como sabemos que os deslocamentos na parede são nulos, então cancelamos as linhas e colunas correspondentes, ficando assim apenas uma matriz reduzida
    matriz_reduzida = matriz_global[2:4, 2:4]

    # Resolvendo o sistema de equações para os deslocamentos nodais desconhecidos
    deslocamentos = matriz_reduzida.LUsolve(vetor_forcas_conhecido)
    return deslocamentos

deslocamentos = resolucao_sistema(matriz_global, vetor_forcas_reduzido)

# Exibindo os deslocamentos nodais desconhecidos
print("Deslocamentos nodais desconhecidos:")
pprint(N(deslocamentos, 2))

Deslocamentos nodais desconhecidos:
⎡-0.00091⎤
⎢        ⎥
⎣-0.0043 ⎦


### Cálculo das Forças de Reação

In [231]:
# Reconstrução do vetor de deslocamentos completo, incluindo os deslocamentos conhecidos (zeros nos apoios)
def calcular_deslocamentos_completos(deslocamentos):
  S = Matrix.zeros(6, 1)
  S[2:4, 0] = deslocamentos
  return S

deslocamentos_completos = calcular_deslocamentos_completos(deslocamentos)

def calcular_forcas_reacao(deslocamentos_completos, matriz_global):
# Cálculo das forças de reação nos apoios utilizando a matriz global de rigidez
  return np.dot(matriz_global, deslocamentos_completos)

forcas_reacao = calcular_forcas_reacao(deslocamentos_completos, matriz_global)

# Exibindo as forças de reação nos apoios
print("Forças de reação nos apoios:")
pprint(N(Matrix(forcas_reacao[0:2])))  # Reações no nó 0
pprint(N(Matrix(forcas_reacao[2:4])))  # Reações no nó 1
pprint(N(Matrix(forcas_reacao[4:6])))  # Reações no nó 2

Forças de reação nos apoios:
⎡-2000.0⎤
⎢       ⎥
⎣1000.0 ⎦
⎡   0   ⎤
⎢       ⎥
⎣-1000.0⎦
⎡2000.0⎤
⎢      ⎥
⎣  0   ⎦


### Transformação para Coordenadas Locais

In [232]:
# Matriz de transformação para o elemento 1
def calcular_matriz_de_rotacao(angulo):
    T = Matrix([
        [cos(angulo), sin(angulo), 0, 0],
        [sin(angulo), cos(angulo), 0, 0],
        [0, 0, cos(angulo), sin(angulo)],
        [0, 0, sin(angulo), cos(angulo)]
    ])
    return T
# Matriz de transformação para o elemento 2
T1 = calcular_matriz_de_rotacao(ang1)
T2 = calcular_matriz_de_rotacao(ang2)

# Transformação dos deslocamentos globais para coordenadas locais
deslocamentos_locais_1 = T1 * deslocamentos_completos[0:4, 0]
deslocamentos_locais_2 = T2 * deslocamentos_completos[2:6, 0]

# Exibindo os deslocamentos transformados para coordenadas locais
print("Deslocamentos em coordenadas locais para o elemento 1:")
pprint(N(deslocamentos_locais_1, 2))

print("\nDeslocamentos em coordenadas locais para o elemento 2:")
pprint(N(deslocamentos_locais_2, 2))

Deslocamentos em coordenadas locais para o elemento 1:
⎡   0   ⎤
⎢       ⎥
⎢   0   ⎥
⎢       ⎥
⎢0.0011 ⎥
⎢       ⎥
⎣-0.0035⎦

Deslocamentos em coordenadas locais para o elemento 2:
⎡-0.00091⎤
⎢        ⎥
⎢-0.0043 ⎥
⎢        ⎥
⎢   0    ⎥
⎢        ⎥
⎣   0    ⎦


### Cálculo das Forças Axiais

In [233]:
# Matriz de forças-deslocamentos local para cálculo das forças axiais
def calcular_forcas_axiais(EAKL, deslocamento_local):
    matriz_forcas_deslocamento = Matrix([
        [1, 0, -1, 0],
        [0, 0, 0, 0],
        [-1, 0, 1, 0],
        [0, 0, 0, 0]
    ])
    return EAKL * matriz_forcas_deslocamento * deslocamento_local

# Cálculo das forças axiais no elemento 1
forcas_axiais_1 = calcular_forcas_axiais(EAKL1, deslocamentos_locais_1)
print("Forças axiais no elemento 1:")
pprint(N(forcas_axiais_1, 7))

# Cálculo das forças axiais no elemento 2
forcas_axiais_2 = calcular_forcas_axiais(EAKL2, deslocamentos_locais_2)
print("\nForças axiais no elemento 2:")
pprint(N(forcas_axiais_2))

Forças axiais no elemento 1:
⎡-2236.068⎤
⎢         ⎥
⎢    0    ⎥
⎢         ⎥
⎢2236.068 ⎥
⎢         ⎥
⎣    0    ⎦

Forças axiais no elemento 2:
⎡-2000.0⎤
⎢       ⎥
⎢   0   ⎥
⎢       ⎥
⎢2000.0 ⎥
⎢       ⎥
⎣   0   ⎦


### Cálculo das Tensões

In [234]:
from sympy import N  # Ensure sympy.N is used for numerical evaluation

# Cálculo das tensões no elemento 1
tensao_1 = (forcas_axiais_1[2, 0] + forcas_axiais_1[3, 0]) / A
print("Tensão no elemento 1 (Pa):")
print(N(tensao_1, 10))
# Cálculo das tensões no elemento 2
tensao_2 = (forcas_axiais_2[0, 0] + forcas_axiais_2[1, 0]) / A
print("\nTensão no elemento 2 (Pa):")
print(N(tensao_2, 10))

Tensão no elemento 1 (Pa):
35493142.50

Tensão no elemento 2 (Pa):
-31746031.75


### Cálculo da Massa e Volume

In [235]:
# Densidade do material (kg/m³)
densidade = 2800

# Volume da seção transversal (m³)
volume1 = A * L1 
volume2 = A * L2
print(f"O volume da barra 1 é: {volume1:.6f} m³")
print(f"O volume da barra 2 é: {volume2:.6f} m³")

massa1 = densidade * volume1
massa2 = densidade * volume2
print(f"A massa da barra 1 é: {massa1:.6f} kg")
print(f"A massa da barra 2 é: {massa2:.6f} kg")

O volume da barra 1 é: 0.000141 m³
O volume da barra 2 é: 0.000126 m³
A massa da barra 1 é: 0.394442 kg
A massa da barra 2 é: 0.352800 kg


# Otimização

In [236]:
# Cálculo do momento de Inércia da seção transversal
I = (0.012**4-0.009**4)/12
pprint(N(I, 6))
# Cálculo da força axial de compressão crítica
Pcr = ((pi**2)*E*I)/(L2**2)
pprint(N(Pcr, 6))

1.18125e-9
204.023


Agora, calcularemos a força axial de compressão crítica com o coeficiente de segurança $1,2$

In [237]:
forca_axial_seguranca = abs(forcas_axiais_2[0, 0]*1.2)
pprint(N(forca_axial_seguranca, 7))

2400.000


$$P_{cr} = \frac{\pi^2EI}{L^2}$$
$$2400 = \frac{\pi^2\times 70 \times 10^9 \times I}{2^2}$$

In [238]:
I = (forca_axial_seguranca * L2**2) / (np.pi**2 * E)

# Convert I to a numerical value for formatting
I_numeric = N(I)
print(f"O valor de I é:", end=" ")
print(N(I, 7), end = " ")
print(f"m^4")

O valor de I é: 1.389548e-8 m^4


$$I = \frac{a^4_e - a^4_i}{12} $$
Reforçarei $a_e$ e manterei $a_i$
$$I = \frac{a^4_e - 0,009^4}{12} $$
Calculando $a_e$

In [239]:
# Calculando a_e
a_e = (12 * I + 0.009**4)**0.25
print(f"O valor de a_e é:", end=" ")
print(N(a_e, 7), end = " ")
print(f"m")

O valor de a_e é: 0.02040346 m


In [243]:
# Atualizando a área da seção transversal com o novo a_e
A = (a_e**2-0.009**2)

# Recalculando as constantes de rigidez axial dos elementos
EAKL1 = E * A / L1
EAKL2 = E * A / L2

# Recalculando as matrizes de rigidez dos elementos
k1 = calcular_matriz_rigidez(E, A, L1, ang1)
k2 = calcular_matriz_rigidez(E, A, L2, ang2)

# Recalculando a matriz global de rigidez
matriz_global = montar_matriz_global(k1, k2)

# Recalculando os deslocamentos nodais desconhecidos
deslocamentos = resolucao_sistema(matriz_global, vetor_forcas_reduzido)

# Recalculando o vetor de deslocamentos completo
deslocamentos_completos = calcular_deslocamentos_completos(deslocamentos)

# Recalculando as forças de reação nos apoios
forcas_reacao = calcular_forcas_reacao(deslocamentos_completos, matriz_global)

# Recalculando os deslocamentos em coordenadas locais
deslocamentos_locais_1 = T1 * deslocamentos_completos[0:4, 0]
deslocamentos_locais_2 = T2 * deslocamentos_completos[2:6, 0]

# Recalculando as forças axiais nos elementos
forcas_axiais_1 = calcular_forcas_axiais(EAKL1, Matrix(deslocamentos_locais_1))
forcas_axiais_2 = calcular_forcas_axiais(EAKL2, Matrix(deslocamentos_locais_2))

# Recalculando as tensões nos elementos
tensao_1 = (forcas_axiais_1[2] + forcas_axiais_1[3]) / A
tensao_2 = (forcas_axiais_2[0] + forcas_axiais_2[1]) / A
# Recalculando a força axial de compressão crítica
Pcr = ((pi**2)*E*I)/(L2**2)

# Exibindo os resultados recalculados
print("Nova área da seção transversal (A):")
pprint(N(A))
print("Nova matriz de rigidez do elemento 1 (k1):")
pprint(N(k1))
print("\nNova matriz de rigidez do elemento 2 (k2):")
pprint(N(k2))
print("\nNova matriz global de rigidez:")
pprint(N(matriz_global))
print("\nNovos deslocamentos nodais desconhecidos:")
pprint(N(deslocamentos))
print("\nNovas forças de reação nos apoios:")
pprint((N(Matrix(forcas_reacao[0:2, 0]))))  # Reações no nó 0
pprint((N(Matrix(forcas_reacao[2:4, 0]))))  # Reações no nó 1
pprint((N(Matrix(forcas_reacao[4:6, 0]))))  # Reações no nó 2
print("\nNovas forças axiais no elemento 1:")
pprint(N(forcas_axiais_1))
print("\nNovas forças axiais no elemento 2:")
pprint(N(forcas_axiais_2))
print("\nNova tensão no elemento 1 (Pa):")
pprint(N(tensao_1))
print("\nNova tensão no elemento 2 (Pa):")
pprint(N(tensao_2))
print("\nNova força axial de compressão crítica:")
pprint(N(Pcr))

Nova área da seção transversal (A):
0.000335301236324943
Nova matriz de rigidez do elemento 1 (k1):
⎡8397271.20245769   -4198635.60122885  -8397271.20245769  4198635.60122885 ⎤
⎢                                                                          ⎥
⎢-4198635.60122885  2099317.80061442   4198635.60122885   -2099317.80061442⎥
⎢                                                                          ⎥
⎢-8397271.20245769  4198635.60122885   8397271.20245769   -4198635.60122885⎥
⎢                                                                          ⎥
⎣4198635.60122885   -2099317.80061442  -4198635.60122885  2099317.80061442 ⎦

Nova matriz de rigidez do elemento 2 (k2):
⎡11735543.271373   0  -11735543.271373  0⎤
⎢                                        ⎥
⎢       0          0         0          0⎥
⎢                                        ⎥
⎢-11735543.271373  0  11735543.271373   0⎥
⎢                                        ⎥
⎣       0          0         0          0⎦

Nova matriz glo

### Tensão de escoamento


A tensão de escoamento com coeficiente de segurança $2$ é dada por

In [241]:
Sy = tensao_1 * 2
pprint(N(Sy))

13337666.1655539


Como podemos ver, a tensão do elemento 1 está dentro da normas de segurança ($\sigma _y = 70 \times 10^6$ Pa)

$$$$

In [242]:
# Densidade do material (kg/m³)
densidade = 2800

# Volume da seção transversal (m³)
volume1 = A * L1 
volume2 = A * L2
print(f"O volume da barra 1 é: {volume1:.6f} m³")
print(f"O volume da barra 2 é: {volume2:.6f} m³")

massa1 = densidade * volume1
massa2 = densidade * volume2
print(f"A massa da barra 1 é: {massa1:.6f} kg")
print(f"A massa da barra 2 é: {massa2:.6f} kg")

O volume da barra 1 é: 0.000750 m³
O volume da barra 2 é: 0.000671 m³
A massa da barra 1 é: 2.099318 kg
A massa da barra 2 é: 1.877687 kg
